<p style = 'color:#E6E4E7 ; background-color:#00561B ; text-align:center ; font-size: 300%'> Kayak project - Get hotel data for Saintes-Maries-de-la-Mer

<p style = 'color:#00561B ; background-color:#E6E4E7 ; text-align:center ; font-size: 150%'> Project prepared by Stephanie Cotineau - #dsmpt-Paris-08

In [1]:
!pip install Scrapy -q

In [2]:
import os # For manipulate file 
import logging # For display logs
import scrapy # Import scrapy & scrapy.crawler
from scrapy.crawler import CrawlerProcess
import re

In [3]:
# Name of the file where the results will be saved
filename = "Saintes_Maries_Mer_multipage.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('results_hotels/'):
        os.remove('results_hotels/' + filename)

# Declare a new CrawlerProcess with some settings
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'
process = CrawlerProcess(settings = {
    'USER_AGENT': user_agent,
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'results_hotels/' + filename : {"format": "json"},
    }
})

In [4]:
from bs4 import BeautifulSoup 
import requests

In [5]:
# Saintes Maries de la Mer - 5 first pages
offset = 0
link_offset = []
while offset != 125:
    url = f"https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AqeVw5QGwAIB0gIkNTY5ODUwZjQtN2RlNC00NTQ4LTlmYjctNTc4MDY3NmJiY2Uy2AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AqeVw5QGwAIB0gIkNTY5ODUwZjQtN2RlNC00NTQ4LTlmYjctNTc4MDY3NmJiY2Uy2AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=saintes+maries+de+la+mer&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&search_pageview_id=12195b135f65001b&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&search_pageview_id=12195b135f65001b&ss_raw=saintes+maries+de+la+mer&offset={offset}"
    #print(url)
    offset = offset + 25
    link_offset.append(url)

In [6]:
link_offset

['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AqeVw5QGwAIB0gIkNTY5ODUwZjQtN2RlNC00NTQ4LTlmYjctNTc4MDY3NmJiY2Uy2AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AqeVw5QGwAIB0gIkNTY5ODUwZjQtN2RlNC00NTQ4LTlmYjctNTc4MDY3NmJiY2Uy2AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=saintes+maries+de+la+mer&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&search_pageview_id=12195b135f65001b&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&search_pageview_id=12195b135f65001b&ss_raw=saintes+maries+de+la+mer&offset=0',
 'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGo

In [7]:
class SaintesSpider(scrapy.Spider) :
    # Name of spider
    name = 'Saintes_Maries_Mer'
    
    # Starting URL
    start_urls = ['https://www.booking.com/']
    
    def start_requests(self):
        for d in link_offset :
            yield scrapy.Request(d,callback=self.search)
    
    # Callback used after search launched
    def search(self, response):
        hotels = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for hotel in hotels:
            yield {
                'location' : hotel.css('span.f4bd0794db.b4273d69aa::text').get(),
                'hotel_url' : hotel.css('a.e13098a59f').attrib['href'],
                'hotel_name' : hotel.css('div.fcab3ed991.a23c043802::text').get(),
                'hotel_rating' : hotel.css('div.b5cd09854e.d10a6220b4::text').get(),
                'hotel_description_short' : hotel.css('div.d8eab2cf7f::text').get()
            }

In [8]:
# Start the crawling using the spider defined above
process.crawl(SaintesSpider)
process.start()

In [9]:
import pandas as pd

In [10]:
saintes = pd.read_json('results_hotels/Saintes_Maries_Mer_multipage.json')

In [11]:
len(saintes)

125

In [12]:
saintes.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short
0,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/residence-les...,Résidence les impériaux,"7,7","Située aux Saintes-Maries-de-la-Mer, la Réside..."
1,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/l-39-etrier.f...,Maison l'Etrier,"9,3","Située aux Saintes-Maries-de-la-Mer, à 1,9 km ..."
2,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/aigrettes-sui...,les Aigrettes 5,"8,9",Hébergement géré par un particulier
3,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/hacienda-des-...,Hôtel restaurant le delta,"6,3","Doté d'un bar, l'Hôtel restaurant le delta est..."
4,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/mas-de-charme...,Mas de charme Les Pellegrins,"9,3",Hébergement géré par un particulier
5,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/appartement-l...,Appartement LE BOIS FLOTTE,"8,8",Hébergement géré par un particulier
6,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/holiday-home-...,Holiday Home L'Etrier Camarguais,"7,3",Le Holiday Home L'Etrier Camarguais est situé ...
7,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/maison-en-cam...,maison en Camargue,"9,4",Hébergement géré par un particulier
8,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/le-gite-de-fa...,Le Gite de Fanny,"8,7",Hébergement géré par un particulier
9,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/el-rocio-1-gi...,EL ROCIO 1 GITES EQUESTRE,"8,9",Hébergement géré par un particulier


In [13]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_ll =[]
for link in saintes['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    lat_lon = soup_link.find_all(id = "hotel_address")
    for ll in lat_lon : 
        #print(ll['data-atlas-latlng'])
        result_ll.append(ll['data-atlas-latlng'])

In [14]:
len(result_ll)

125

In [15]:
saintes['Latitude_Longitude'] = result_ll

In [16]:
lat = []
lon = []

# For each row in a varible,
for row in saintes['Latitude_Longitude']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
saintes['latitude'] = lat
saintes['longitude'] = lon

In [17]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_long_description =[]
for link in saintes['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    long_description = soup_link.find_all(id = "property_description_content") 
    for ld in long_description : 
        result_long_description.append(ld.text)

In [18]:
len(result_long_description)

125

In [19]:
saintes['hotel_description_long'] = result_long_description

In [20]:
saintes.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/residence-les...,Résidence les impériaux,"7,7","Située aux Saintes-Maries-de-la-Mer, la Réside...","43.45465520,4.42902280",43.45465520,4.42902280,"\nSituée aux Saintes-Maries-de-la-Mer, la Rési..."
1,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/l-39-etrier.f...,Maison l'Etrier,"9,3","Située aux Saintes-Maries-de-la-Mer, à 1,9 km ...","43.46729600,4.41386700",43.46729600,4.41386700,\nCet établissement est à 18 minutes à pied de...
2,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/aigrettes-sui...,les Aigrettes 5,"8,9",Hébergement géré par un particulier,"43.45444200,4.43084200",43.45444200,4.43084200,\nCet établissement est à 4 minutes à pied de ...
3,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/hacienda-des-...,Hôtel restaurant le delta,"6,3","Doté d'un bar, l'Hôtel restaurant le delta est...","43.45217800,4.42811600",43.45217800,4.42811600,\nCet établissement est à 3 minutes à pied de ...
4,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/mas-de-charme...,Mas de charme Les Pellegrins,"9,3",Hébergement géré par un particulier,"43.48681127,4.40356759",43.48681127,4.40356759,\nDoté d'une piscine extérieure ouverte en sai...
5,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/appartement-l...,Appartement LE BOIS FLOTTE,"8,8",Hébergement géré par un particulier,"43.45064690,4.42469990",43.45064690,4.42469990,\nCet établissement est à 3 minutes à pied de ...
6,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/holiday-home-...,Holiday Home L'Etrier Camarguais,"7,3",Le Holiday Home L'Etrier Camarguais est situé ...,"43.46834142,4.41216458",43.46834142,4.41216458,\nLe Holiday Home L'Etrier Camarguais est situ...
7,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/maison-en-cam...,maison en Camargue,"9,4",Hébergement géré par un particulier,"43.46825893,4.41530266",43.46825893,4.41530266,"\nSituée aux Saintes-Maries-de-la-Mer, à 2 km ..."
8,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/le-gite-de-fa...,Le Gite de Fanny,"8,7",Hébergement géré par un particulier,"43.46801240,4.41265190",43.46801240,4.41265190,"\nSitué aux Saintes-Maries-de-la-Mer, à moins ..."
9,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/el-rocio-1-gi...,EL ROCIO 1 GITES EQUESTRE,"8,9",Hébergement géré par un particulier,"43.50288670,4.39302386",43.50288670,4.39302386,"\nOffrant une vue sur le jardin, l'EL ROCIO 1 ..."


In [21]:
saintes['hotel_description_long'] = saintes['hotel_description_long'].replace('\n','', regex=True)

In [22]:
saintes.head(5)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/residence-les...,Résidence les impériaux,"7,7","Située aux Saintes-Maries-de-la-Mer, la Réside...","43.45465520,4.42902280",43.45465520,4.42902280,"Située aux Saintes-Maries-de-la-Mer, la Réside..."
1,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/l-39-etrier.f...,Maison l'Etrier,"9,3","Située aux Saintes-Maries-de-la-Mer, à 1,9 km ...","43.46729600,4.41386700",43.46729600,4.41386700,Cet établissement est à 18 minutes à pied de l...
2,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/aigrettes-sui...,les Aigrettes 5,"8,9",Hébergement géré par un particulier,"43.45444200,4.43084200",43.45444200,4.43084200,Cet établissement est à 4 minutes à pied de la...
3,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/hacienda-des-...,Hôtel restaurant le delta,"6,3","Doté d'un bar, l'Hôtel restaurant le delta est...","43.45217800,4.42811600",43.45217800,4.42811600,Cet établissement est à 3 minutes à pied de la...
4,Les Saintes-Maries-de-la-Mer,https://www.booking.com/hotel/fr/mas-de-charme...,Mas de charme Les Pellegrins,"9,3",Hébergement géré par un particulier,"43.48681127,4.40356759",43.48681127,4.40356759,Doté d'une piscine extérieure ouverte en saiso...


In [23]:
saintes['hotel_description_long'][55]

"Occupant une maison de campagne typiquement camarguaise entourée d’un parc de 250 hectares où vivent 200 taureaux de Camargue, le Manade Cavallini - Mas de Pioch propose des chambres et des appartements aux Saintes-Maries-de-la-Mer. Il possède une grande terrasse meublée, un jardin ombragé et une piscine extérieure.Les chambres se trouvent au premier étage et les appartements au rez-de-chaussée, à 1,5 km du centre de l’établissement. Tous les logements comprennent une salle de bains privative pourvue d'une baignoire ou d'une douche. Certains disposent d’une télévision.Un petit-déjeuner français composé de pain, de viennoiseries, de jus de fruits et de boissons chaudes est servi dans la salle à manger, sur la terrasse ou dans le jardin.Un service de location de vélos et des sorties à cheval peuvent être organisés sur place moyennant des frais supplémentaires et sur demande effectuée au moins 24 heures à l’avance. Vous pourrez bénéficier de tarifs préférentiels sur certaines activités. 

In [24]:
# Clean location column
saintes['location'] = ["Saintes" if value!="Saintes" else value for value in saintes["location"]]

In [25]:
saintes.head()

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Saintes,https://www.booking.com/hotel/fr/residence-les...,Résidence les impériaux,"7,7","Située aux Saintes-Maries-de-la-Mer, la Réside...","43.45465520,4.42902280",43.45465520,4.42902280,"Située aux Saintes-Maries-de-la-Mer, la Réside..."
1,Saintes,https://www.booking.com/hotel/fr/l-39-etrier.f...,Maison l'Etrier,"9,3","Située aux Saintes-Maries-de-la-Mer, à 1,9 km ...","43.46729600,4.41386700",43.46729600,4.41386700,Cet établissement est à 18 minutes à pied de l...
2,Saintes,https://www.booking.com/hotel/fr/aigrettes-sui...,les Aigrettes 5,"8,9",Hébergement géré par un particulier,"43.45444200,4.43084200",43.45444200,4.43084200,Cet établissement est à 4 minutes à pied de la...
3,Saintes,https://www.booking.com/hotel/fr/hacienda-des-...,Hôtel restaurant le delta,"6,3","Doté d'un bar, l'Hôtel restaurant le delta est...","43.45217800,4.42811600",43.45217800,4.42811600,Cet établissement est à 3 minutes à pied de la...
4,Saintes,https://www.booking.com/hotel/fr/mas-de-charme...,Mas de charme Les Pellegrins,"9,3",Hébergement géré par un particulier,"43.48681127,4.40356759",43.48681127,4.40356759,Doté d'une piscine extérieure ouverte en saiso...


In [26]:
saintes.to_csv("Saintes_Maries_Mer.csv")